In [14]:
import psycopg2
import pandas as pd

In [19]:
host = '172.25.80.1'
dbname = 'postgres'
user = 'postgres'
pwd = 'admin'
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
postgres_insert_query = """ select * from view_books """

cur.execute(postgres_insert_query)

cur.fetchall()

#postgresql 연결종료
conn.cursor().close()
conn.close()


In [15]:

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

#데이터 가져오기
command = "select distinct title, date, url, hashtag from yt_clothes"
cur.execute(command)
conn.commit()
data = cur.fetchall()

#DF화
df = pd.DataFrame(data)
df.columns = ['title', 'date', 'url', 'hashtag']

#db 연결 종료
conn.cursor().close()
conn.close()


select distinct title, date, url, hashtag from yt_clothes


In [16]:
df.head()

,title,date,url,hashtag
0,아...니들스 로고 떼고싶다...,2022-02-11,https://www.youtube.com/watch?v=k9cAUZ395D4,#니들스 #셔츠 #아메카지 #스컬프스토어 #후기 #리뷰
1,📌68kg 피팅모델 체형커버 코디룩북📌 outfitㅣlook-bookㅣ66사이즈ㅣ7...,2022-03-12,https://www.youtube.com/watch?v=ascmsevZpHY,#룩북 #통통녀
2,잠옷바지 홈웨어 찰랑거리는 소재감,2021-09-21,https://www.youtube.com/shorts/Ixycw20dHfw,
3,치노 와이드 밴딩 코튼 팬츠,2021-09-01,https://www.youtube.com/shorts/Nlp4JSSyWPo,
4,유니클로에서 출시한 역대급 데님팬츠,2022-02-16,https://www.youtube.com/watch?v=P4sBaidETAo,#유니클로 #데님팬츠 #청바지


In [17]:
#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

In [ ]:
#postgresql 접속
conn_string = "host='kedu.postgres.database.azure.com' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_clothes'

#table에 데이터 넣기
for i in range(len(df)) :
    cur.execute(insert_command(table), (df.iloc[i][0], df.iloc[i][1], df.iloc[i][2], df.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()

In [6]:
cur.execute("select * from test2")
aa

[('샴푸', '13200', '85000', 49.5, 679.3, 0.41, 0.87, 15, '높음'),
 ('유아입욕제', '210', '1310', 1.1, 21.0, 0.57, 1.67, 15, '높음'),
 ('아기샴푸', '430', '3380', 1.8, 32.0, 0.46, 1.02, 15, '높음'),
 ('약산성샴푸', '2090', '23600', 5.5, 225.4, 0.29, 1.03, 15, '높음'),
 ('아기로션', '1600', '20800', 2.7, 3.3, 0.19, 0.02, 15, '높음'),
 ('유아거품목욕', '100', '900', 0.5, 21.3, 0.6, 2.45, 15, '높음'),
 ('탈모샴푸', '13700', '67300', 41.0, 520.2, 0.34, 0.82, 15, '높음'),
 ('아기바디워시', '1830', '9580', 1.0, 62.3, 0.07, 0.69, 15, '높음'),
 ('천연샴푸', '1940', '17700', 7.7, 138.7, 0.42, 0.86, 15, '높음'),
 ('아기거품목욕', '270', '2790', 1.2, 67.7, 0.48, 2.61, 15, '높음'),
 ('버블바스', '200', '1440', 1.6, 21.8, 0.85, 1.58, 15, '높음'),
 ('유아바디워시', '570', '14300', 2.1, 0.7, 0.4, 0.01, 15, '높음'),
 ('아기입욕제', '1940', '1850', 1.1, 20.1, 0.06, 1.15, 15, '높음'),
 ('거품목욕', '250', '1670', 0.3, 19.8, 0.15, 1.27, 15, '높음'),
 ('촉감놀이', '820', '3660', 3.1, 5.8, 0.4, 0.17, 15, '높음'),
 ('크리스마스선물', '260', '1040', 0.2, 0.3, 0.11, 0.04, 12, '중간'),
 ('브로콜리샴푸', '10', '40', 0.0, 0.

# 유튜브 DB 뽑아서 해시태그 추가하는 작업

- 4000개 크롤링 완료 >> db 적재 >> select문으로 남은놈들 데려와서 다시 한번 돌리고 >> 담부턴 찾는대로 넣을 수 있게 해야지

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [5]:
#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

In [126]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_clothes'

#table 확인하기
cur.execute('select * from yt_clothes where id < 5952 order by id')
conn.commit()
datalist = cur.fetchall()


In [128]:
dataDf = pd.DataFrame(datalist)
dataDf.columns = ['id', 'title', 'date', 'url', 'hashtag']

In [129]:
dataDf.shape

(5951, 5)

In [ ]:
dataDf.head()

In [133]:
url_list = dataDf['url'].tolist()
print(len(url_list))

hashtag_list = [] #해시태그

5951


In [138]:
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        print(hash_tag)
        hashtag_list.append(hash_tag)

In [219]:
driver = webdriver.Chrome('C:\chromedriver.exe')

#해시태그 가져오기
for i in range(5021, len(url_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(3)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        # 해시태그 가져오기
        outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
        get_hashtag(outline_test)
        

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(3)  

        # 해시태그 가져오기
        outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
        get_hashtag(outline_test)

print('끝')

5022 : 일반 #유니클로 #유니클로
5023 : 쇼츠 #쇼츠
5024 : 쇼츠 
5025 : 일반 #겨울패션하울 #맨투맨추천 #니트추천
5026 : 쇼츠 
5027 : 쇼츠 
5028 : 일반 
5029 : 일반 #남자니트집업 #남자니트집업추천 #남자니트집업가격 #남자니트집업후기 #가성비남자니트집업 #남자니트집업순위 #아도르클래식 #브로기모니트집업 #코디코치 #닥스 #다꾸앙 #맨타임 #단군 #맨타임
5030 : 일반 
5031 : 일반 
5032 : 쇼츠 
5033 : 쇼츠 
5034 : 일반 #남자니트집업 #남자니트집업추천 #남자니트집업가격 #남자니트집업후기 #가성비남자니트집업 #남자니트집업순위 #진마니아 #코디코치 #닥스 #다꾸앙 #맨타임 #단군 #맨타임
5035 : 쇼츠 
5036 : 쇼츠 
5037 : 일반 
5038 : 일반 
5039 : 쇼츠 
5040 : 쇼츠 
5041 : 쇼츠 #남자후드집업 #스판후드 #투웨이자켓 #남자플리츠집업 #플리츠후드집업
5042 : 쇼츠 
5043 : 일반 #앨빈클로 #빌리언즈 #아르메데스 #맨타임 #그라픽 #오티디 #듀클라이 #매드문 #남녀공용 #댕글드랍
5044 : 일반 #스웨트 #스웨트셋업 #멜란지마스터
5045 : 일반 #숏패딩 #패딩코디 #남자겨울코디
5046 : 일반 
5047 : 쇼츠 
5048 : 일반 #여름아우터 #린넨 #여름코디
5049 : 일반 #남자여름코디 #남자봄코디 #남자반팔티
5050 : 일반 
5051 : 일반 #여름아우터 #여름코디 #린넨
5052 : 일반 
5053 : 일반 #코디 #초등학생 #초딩
5054 : 일반 #긴팔추천 #더죽긴 #여름긴팔
5055 : 일반 #남자여름옷 #남자반팔티
5056 : 일반 #스스므 #테스터스
5057 : 일반 #기본템 #여름기본템
5058 : 일반 #남자여름코디 #남자여름옷 #반팔티
5059 : 일반 #남자여름코디 #여름필수템 #치트키아이템
5060 : 일반 
5061 : 일반 #코디공식 #패션꿀팁 #패션노하우 #패알못 #패션공식 #여름코디 #패션 #

In [220]:
len(hashtag_list)

5951

In [234]:
len(url_list)

5951

In [218]:
print(url_list[5020], ' / ', hashtag_list[5950])

https://www.youtube.com/watch?v=eGHGedHwliU  /  


In [217]:
hashtag_list.append('')

In [231]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

#hashtag_list[1224] = hashtag_list[1224][:203]

0 18
2 112
19 247
274 357
605 380


In [230]:
hashtag_list[3415] = hashtag_list[3415][:205]

In [232]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [235]:
#insert문 만들기
def update_command(table, i) :
    command = 'update ' + table + ' set hashtag = %s where id = %s ;'
    return command

table = 'yt_clothes'

#table에 데이터 넣기
for i in range(len(hashtag_list)) :
    cur.execute(update_command(table, i), (hashtag_list[i], i+1))
    conn.commit()